In [2]:
!ipython nbconvert --to rst tutorial.ipynb


[TerminalIPythonApp] WARNING | Subcommand `ipython nbconvert` is deprecated and will be removed in future versions.
[TerminalIPythonApp] WARNING | You likely want to use `jupyter nbconvert` in the future
[NbConvertApp] Converting notebook tutorial.ipynb to rst
[NbConvertApp] Writing 52500 bytes to tutorial.rst


# Quick-start
## Install datatable

Let’s begin by installing the latest stable version of ``datatable`` from PyPI:


~~~ bash
$ pip install datatable
~~~

If this didn’t work for you, or if you want to install the bleeding edge version of the library, please check the [Installation page](install.rst)

Assuming the installation was successful, you can now import the library in a JupyterLab notebook or in a Python console:

In [1]:
import datatable as dt

dt.__version__

'0.8.0'

## Create Frame

The fundamental unit of analysis in datatable is a ``Frame``. It is the same notion as a pandas DataFrame or SQL table: data arranged in a two-dimensional array with rows and columns.

You can create a ``Frame`` object from a variety of data sources:

* from a python **list** or **dictionary**:

In [ ]:
import datatable as dt
import math

DT = dt.Frame(A=range(5), B=[1.7, 3.4, 0, None, -math.inf], 
              C = ['two','one','one','two','two'],
                 stypes={"A": dt.int64})
DT

,A,B,C
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,0,1.7,two
1,1,3.4,one
2,2,0,one
3,3,NA,two
4,4,−inf,two


* from a **numpy array**

In [3]:
import numpy as np

np.random.seed(1)
DT2 = dt.Frame(np.random.randn(3))
DT2

,C0
,▪▪▪▪▪▪▪▪
0,1.62435
1,−0.611756
2,−0.528172


* from a **pandas DataFrame**.

In [4]:
import pandas as pd

DT3 = dt.Frame(pd.DataFrame({"A": range(3)}))
DT3

,A
,▪▪▪▪▪▪▪▪
0,0
1,1
2,2


## Convert Frame

Convert an existing ``Frame`` into a numpy array, a pandas DataFrame - requires ``pandas`` and ``numpy``:

In [5]:
DT_numpy = DT.to_numpy()
DT_pandas = DT.to_pandas()

A frame can also be converted into python native data structures: a dictionary, keyed by the column names; a list of columns, where each column is itself a list of values; or a list of rows, where each row is a tuple of values:

In [6]:
DT_list = DT.to_list()
DT_dict = DT.to_dict()
DT_tuple = DT.to_tuples()

## Read data

You can also load a CSV/text/Excel file, or open a previously saved binary ``.jay`` file:

~~~ python
DT4 = dt.fread("dataset_01.xlsx")
DT5 = dt.fread("dataset_02.csv")
DT6 = dt.open("data.jay")
~~~

``fread()`` function shown above is both powerful and extremely fast. It can automatically detect parse parameters for the majority of text files, load data from .zip archives or URLs, read Excel files, and much more.

* Automatically detects separators, headers, column types, quoting rules, etc.
* Reads from majority of text files, load data from .zip archives or URLs, read Excel files, URL, shell, raw text, * archives, glob
* Provides multi-threaded file reading for maximum speed
* Includes a progress indicator when reading large files
* Reads both RFC4180-compliant and non-compliant files

## Write data

Write the Frame’s content into a ``.csv`` file in a multi-threaded way:

~~~ python

DT.to_csv("out.csv")
~~~

You can also save a frame into a binary ``.jay`` file:

~~~ python

DT.to_jay("data.jay")
~~~

## Frame Properties

Investigate your Frame using descripting operators

In [7]:
DT.shape # number of rows and columns

(5, 3)

In [8]:
DT.names # column names

('A', 'B', 'C')

In [9]:
DT.stypes # column types

(stype.int64, stype.float64, stype.str32)

## Data manipulation

Once the data is loaded into a Frame, you may want to do certain operations with it: extract/remove/modify subsets of the data, perform calculations, reshape, group, join with other datasets, etc. In datatable, the primary vehicle for all these operations is the square-bracket notation inspired by traditional matrix indexing but overcharged with power (this notation was pioneered in R data.table and is the main axis of intersection between these two libraries).

In short, almost all operations with a Frame can be expressed as:

where ``i`` is the row selector, ``j`` is the column selector, and ``...`` indicates
that additional modifiers might be added. If this looks familiar to you,
that's because it is. Exactly the same ``DT[i, j]`` notation is used in
mathematics when indexing matrices, in C/C++, in R, in pandas, in numpy, etc.
The only difference that datatable introduces is that it allows
``i`` to be anything that can conceivably be
interpreted as a row selector: an integer to select just one row, a slice,
a range, a list of integers, a list of slices, an expression, a boolean-valued
Frame, an integer-valued Frame, an integer numpy array, a generator, and so on.

The ``j`` column selector is even more versatile.
In the simplest case, you can select just a single column by its index or name. But
also accepted are a list of columns, a slice, a string slice (of the form ``"A":"Z"``), a
list of booleans indicating which columns to pick, an expression, a list of
expressions, and a dictionary of expressions. (The keys will be used as new names
for the columns being selected.) The ``j``
expression can even be a python type (such as ``int`` or ``dt.float32``),
selecting all columns matching that type.

In [10]:
DT[:, "A"]         # select 1 column

,A
,▪▪▪▪▪▪▪▪
0,0
1,1
2,2
3,3
4,4


In [11]:
DT[:3, :]         # first 3 rows

,A,B,C
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,0,1.7,two
1,1,3.4,one
2,2,0,one


In [12]:
DT[::-1, "A":"C"]  # reverse rows order, columns from A to C

,A,B,C
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,4,−inf,two
1,3,NA,two
2,2,0,one
3,1,3.4,one
4,0,1.7,two


In [13]:
DT[3, 2]          # single element in row 3, column 2 (0-based)

'two'

In addition to the selector expression shown above, we support the update and
delete statements too:

~~~ python
  DT[i, j] = r
  del DT[i, j]
~~~

The first expression will replace values in the subset ``[i, j]`` of Frame
``DT`` with the values from ``r``, which could be either a constant, or a
suitably-sized Frame, or an expression that operates on frame ``DT``.

The second expression deletes values in the subset ``[i, j]``. This is
interpreted as follows: if ``i`` selects all rows, then the columns given by
``j`` are removed from the Frame; if ``j`` selects all columns, then the rows
given by ``i`` are removed; if neither ``i`` nor ``j`` span all rows/columns
of the Frame, then the elements in the subset ``[i, j]`` are replaced with
NAs.

In [14]:
DT[:,"X"] = 53    # create new column and assign it value   
DT

,A,B,C,X
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪
0,0,1.7,two,53
1,1,3.4,one,53
2,2,0,one,53
3,3,NA,two,53
4,4,−inf,two,53


In [15]:
DT[1:3,["X","Z"]] = 55  # update existing and create new column with new value
DT

,A,B,C,X,Z
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪,▪
0,0,1.7,two,53,NA
1,1,3.4,one,55,55
2,2,0,one,55,55
3,3,NA,two,53,NA
4,4,−inf,two,53,NA


In [16]:
del DT[:,"X"]
DT

,A,B,C,Z
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪
0,0,1.7,two,NA
1,1,3.4,one,55
2,2,0,one,55
3,3,NA,two,NA
4,4,−inf,two,NA


## Compute Per-Column Summary Stats

Detailed description of Frame functions can be found in [Frame documentation](api/frame.rst)

In [17]:
DT.sum()

,A,B,C,Z
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,10,−inf,NA,110


In [18]:
DT.max()

,A,B,C,Z
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪
0,4,3.4,NA,55


In [19]:
DT.min()

,A,B,C,Z
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪
0,0,−inf,NA,55


In [20]:
DT.mean()

,A,B,C,Z
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,2,−inf,NA,55


In [21]:
DT.sd()

,A,B,C,Z
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1.58114,NA,NA,0


In [22]:
DT.mode()

,A,B,C,Z
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪
0,0,−inf,two,55


In [23]:
DT.nmodal()

,A,B,C,Z
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,1,3,2


In [24]:
DT.nunique()

,A,B,C,Z
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,5,4,2,1


## What the f.?


You may have noticed already that we mentioned several times the possibility
of using expressions in ``i`` or
``j`` and in other places. In the simplest form
an expression looks like

~~~python
  f.ColA
~~~

which indicates a column ``ColA`` in some Frame. Here ``f`` is a variable that
has to be imported from the datatable module. This variable provides a convenient
way to reference any column in a Frame. In addition to the notation above, the
following is also supported:

~~~python
  f[3]
  f["ColB"]
~~~

denoting the fourth column and the column ``ColB`` respectively.

Compute columnar expressions using:

~~~python
df[:, {"x": f.x, "y": f.y, "x+y": f.x + f.y, "x-y": f.x - f.y}]
~~~

These f-expressions support arithmetic operations as well as various mathematical and
aggregate functions. For example, in order to select the values from column
``A`` normalized to range ``[0; 1]`` we can write the following:

In [25]:
from datatable import f, min, max

DT[:, {"A_normalized":(f.A - min(f.A))/(max(f.A) - min(f.A))}]

,A_normalized
,▪▪▪▪▪▪▪▪
0,0
1,0.25
2,0.5
3,0.75
4,1


This is equivalent to the following SQL query:

~~~ SQL
  SELECT (f.A - MIN(f.A))/(MAX(f.A) - MIN(f.A)) FROM DT AS f
~~~

So, what exactly is ``f``? We call it a "**frame proxy**", as it becomes a
simple way to refer to the Frame that we currently operate on. More precisely,
whenever ``DT[i, j]`` is evaluated and we encounter an ``f``-expression there,
that ``f`` becomes replaced with the frame ``DT``, and the columns are looked
up on that Frame. The same expression can later on be applied to a different
Frame, and it will refer to the columns in that other Frame.

At some point you may notice that that datatable also exports symbol ``g``. This
``g`` is also a frame proxy; however it already refers to the *second* frame in
the evaluated expression. This second frame appears when you are *joining* two
or more frames together (more on that later). When that happens, symbol ``g`` is
used to refer to the columns of the joined frame.

This syntax allows do comlex filtering in user friendly way:

In [26]:
DT[f.A > 1,"A":"B"]  # conditional selecting

,A,B
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,2,0
1,3,NA
2,4,−inf


In [27]:
from datatable import sd, mean

DT[(f.A > mean(f.B) + 2.5 * sd(f.A)) | (f.A < -mean(f.Z) - sd(f.B)), #which rows to select
   ["A","C"]] #which columns to select

,A,C
,▪▪▪▪▪▪▪▪,▪▪▪▪
0,0,two
1,1,one
2,2,one
3,3,two
4,4,two


## Groupbys / joins

In the `Data Manipulation`_ section we mentioned that the ``DT[i, j, ...]`` selector
can take zero or more modifiers, which we denoted as ``[...]``. The available
modifiers are ``by()``, ``join()`` and ``sort()``. Thus, the full form of the
square-bracket selector is:

### by(...)

This modifier splits the frame into groups by the provided column(s), and then
applies ``i`` and ``j`` within
each group. This mostly affects aggregator functions such as ``sum()``,
``min()`` or ``sd()``, but may also apply in other circumstances. For example,
if ``i`` is a slice that takes the first 5 rows of a frame,
then in the presence of the ``by()`` modifier it will take the first 5 rows of
each group.

For example, in order to find the total amount of each product sold, write:

In [28]:
from datatable import f, by, sum

DT[:, {"sum_A":sum(f.A)}, by(f.C)]

,C,sum_A
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,one,3
1,two,7


or calculate mean value by groups in colums

In [29]:
from datatable import mean

DT[:, {"mean_A" : mean(f.A)}, by("C")]

,C,mean_A
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,one,1.5
1,two,2.33333


### sort(...)

This modifier controls the order of the rows in the result, much like SQL clause
``ORDER BY``. If used in conjunction with ``by()``, it will order the rows
within each group.

In [30]:
from datatable import sort

DT[:,:,sort(f.B)]

,A,B,C,Z
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪
0,3,NA,two,NA
1,4,−inf,two,NA
2,2,0,one,55
3,0,1.7,two,NA
4,1,3.4,one,55


In [31]:
DT.sort("Z")

,A,B,C,Z
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪
0,0,1.7,two,NA
1,3,NA,two,NA
2,4,−inf,two,NA
3,1,3.4,one,55
4,2,0,one,55


### join(...)


As the name suggests, this operator allows you to join another frame to the
current, equivalent to the SQL ``JOIN`` operator. Currently we support only
left outer joins.

In order to join frame ``X``, it must be keyed. A keyed frame is conceptually
similar to a SQL table with a unique primary key. This key may be either a
single column, or several columns:

~~~ python
X.key = "id"
~~~

Once a frame is keyed, it can be joined to another frame ``DT``, provided that
``DT`` has the column(s) with the same name(s) as the key in ``X``:

~~~ python
DT[:, :, join(X)]
~~~ 

This has the semantics of a natural left outer join. The ``X`` frame can be
considered as a dictionary, where the key column contains the keys, and all
other columns are the corresponding values. Then during the join each row of
``DT`` will be matched against the row of ``X`` with the same value of the
key column, and if there are no such value in ``X``, with an all-NA row.

The columns of the joined frame can be used in expressions using the ``g.``
prefix.

**NOTE:** In the future, we will expand the syntax of the join operator to allow other kinds of joins and also to remove the limitation that only keyed frames can be joined.

In [32]:
DT1 = dt.Frame(product_id = [1, 1, 1, 2, 2, 2, 3, 3, 3], 
               quantity = [11, 22, 16, 45, 65, 60, 33, 37, 39], 
               stypes={"quantity": dt.int64})
DT1

,product_id,quantity
,▪,▪▪▪▪▪▪▪▪
0,1,11
1,1,22
2,1,16
3,2,45
4,2,65
5,2,60
6,3,33
7,3,37
8,3,39


In [33]:
DT2 = dt.Frame(product_id = [1, 2, 3], price = [1, 2, 3], 
               stypes={"price": dt.int64})
DT2

,product_id,price
,▪,▪▪▪▪▪▪▪▪
0,1,1
1,2,2
2,3,3


In [34]:
from datatable import g, join

DT2.key = "product_id"
DT3 = DT1[:, {"sales": f.quantity * g.price}, by(f.product_id), join(DT2)]
DT3

,product_id,sales
,▪,▪▪▪▪▪▪▪▪
0,1,11
1,1,22
2,1,16
3,2,90
4,2,130
5,2,120
6,3,99
7,3,111
8,3,117


## Append 

Append rows / columns to a Frame using:

~~~python
df1.cbind(df2, df3)
df1.rbind(df4, force = True)
~~~

In [35]:
DT1.cbind(DT3[:,"sales"])
DT1

,product_id,quantity,sales
,▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,11,11
1,1,22,22
2,1,16,16
3,2,45,90
4,2,65,130
5,2,60,120
6,3,33,99
7,3,37,111
8,3,39,117


In [36]:
DT1.rbind(DT, force = True)
DT1

,product_id,quantity,sales,A,B,C,Z
,▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪
0,1,11,11,NA,NA,NA,NA
1,1,22,22,NA,NA,NA,NA
2,1,16,16,NA,NA,NA,NA
3,2,45,90,NA,NA,NA,NA
4,2,65,130,NA,NA,NA,NA
5,2,60,120,NA,NA,NA,NA
6,3,33,99,NA,NA,NA,NA
7,3,37,111,NA,NA,NA,NA
8,3,39,117,NA,NA,NA,NA
